In [ ]:
import sys
sys.path.append('../')
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

import torch
import numpy as np
from tqdm import tqdm
from torchvision import datasets, transforms

from Network.ReActNet_18_Qaw import *
from Network.ReActNet_A_Qaw import *
from Network.utils import *

In [ ]:
model = 'resnet18'
# model = 'resnet34'
# model = 'resneta'

In [ ]:
Dataset_path = '/ssd/Datasets/ImageNet/'
Batch_size = 256
Workers = 8
Targetnum = 1000

In [ ]:
_seed_ = 2023
torch.manual_seed(_seed_)
np.random.seed(_seed_)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0)),
    Lighting(0.1),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])

transform_test = transforms.Compose([                      
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
])

Train_data = datasets.ImageFolder(root=Dataset_path+'train', transform=transform_train)
Test_data = datasets.ImageFolder(root=Dataset_path+'val', transform=transform_test)

train_data_loader = torch.utils.data.DataLoader(
    dataset=Train_data,
    batch_size=Batch_size,
    shuffle=True,
    num_workers=Workers, 
    pin_memory=True,
    drop_last=True
)

test_data_loader = torch.utils.data.DataLoader(
    dataset=Test_data,
    batch_size=Batch_size,
    shuffle=False,
    num_workers=Workers, 
    pin_memory=True,
    drop_last=False
)

In [ ]:
if model == 'resnet18':
    net = ResNet18(num_classes=Targetnum, imagenet=True)
    net_model_path = './models/imagenet_reactnet18_step2.h5'
elif model == 'resnet34':
    net = ResNet34(num_classes=Targetnum, imagenet=True)
    net_model_path = './models/imagenet_reactnet34_step2.h5'
elif model == 'resneta':
    net = Reactnet(num_classes=Targetnum, imagenet=True)
    net_model_path = './models/imagenet_reactneta_step2.h5'
else:
    raise ValueError('model error')

net = torch.nn.DataParallel(net).cuda()
net.load_state_dict(torch.load(net_model_path, weights_only=False))

In [ ]:
Confusion_Matrix = torch.zeros((Targetnum, Targetnum))
net.eval()
with torch.no_grad():
    for img, label in tqdm(test_data_loader):
        img = img.cuda()
        label = label.cuda()
        out_fr = net(img)
        guess = out_fr.argmax(1)
        for j in range(len(label)):
            Confusion_Matrix[label[j],guess[j]] += 1
acc = Confusion_Matrix.diag()
acc = acc.sum()/Confusion_Matrix.sum()
print(f'Confusion_Matrix = \n{Confusion_Matrix}')
print(f'acc = {acc}')